# Integrated Intelligent Systems: Module 10 - Cognitive Architectures (Tutorial)

This notebook introduces the fundamentals of cognitive architectures for autonomous robots and basic practices of it.

## Part 1: Concepts and Definitions

In [1]:
from IPython.display import display, HTML,IFrame
display(IFrame('ca_quiz.html', width=1000, height=1350))

## Part 2: Hands-On (Applications of Cognitive Architectures )

To prepare you for the Autonomous Navigate-to-Grasp Challenge, these three Python-based exercises focus on the three core cognitive architecture types (cognitivist, emergent, hybrid).

![Cognitive Architectures](imgs/ca1.jpg "Cognitive Architecture" )

### Exercise 2.1: Emergent Architectures (Subsumption / Neural Networks) 

In the subsumption architecture, intelligence arises from the interactions of a hierarchical stack of behaviors where the most critical "survival" behaviors are at the bottom (closest to the motor) so they can subsume the more complex "luxury" goals above them.

- Layer 0 (Avoidance): Highest Priority. Must be able to override everything to prevent a crash.

- Layer 1 (Battery): High Priority. If power is critical, it must stop the robot from wandering, but it shouldn't disable the "Avoidance" reflex.

- Layer 2 (Navigate): Medium Priority. A specific goal-directed behavior (e.g., go to a coordinate).

- Layer 3 (Wander): Lowest Priority. Only happens if there is no specific mission and the battery is fine.

![Emergent Architectures](imgs/emergent1.jpg "Emergent Architectures" )

In [6]:
class SubsumptionRobot:
    def __init__(self):
        # Internal States (The "Body")
        self.battery_level = 0.05  # 5% battery
        self.target_location = "Kitchen"
        self.sensor_dist = 0.3     # 0.3 meters to a wall

    # --- BEHAVIOR LAYERS ---
    
    def layer_0_avoid(self):
        """Reflex: If too close, turn."""
        if self.sensor_dist < 0.5:
            return "ACTION: Emergency Turn (90 degrees)"
        return None

    def layer_1_battery(self):
        """Inhibitor: If battery low, stop all movement to conserve."""
        if self.battery_level < 0.10:
            return "INHIBIT" # Signal to stop higher layers
        return None

    def layer_2_navigate(self):
        """Goal: Head toward a specific location."""
        return f"ACTION: Moving toward {self.target_location}"

    def layer_3_wander(self):
        """Default: Random exploration."""
        return "ACTION: Wandering randomly"

    # --- THE ARCHITECTURE (The Invariant Switchboard) ---
    
    def execute_cycle(self):
        # 1. Gather signals from all layers
        avoid = self.layer_0_avoid()
        battery_inhibit = self.layer_1_battery()
        navigate = self.layer_2_navigate()
        wander = self.layer_3_wander()

        # 2. Apply Subsumption Logic (Top-Down Inhibition/Suppression)
        
        # MISSION LEVEL: Navigation suppresses Wandering
        # (If we have a destination, we don't wander)
        current_intent = navigate if navigate else wander

        # SAFETY LEVEL: Battery inhibits the current intent
        if battery_inhibit == "INHIBIT":
            print("[System] Battery Layer is INHIBITING Navigation/Wander.")
            current_intent = "IDLE (Conserving Power)"

        # SURVIVAL LEVEL: Avoidance SUBSUMES (Suppresses) everything else
        # Even if the battery is low or we are idle, if we are about to hit
        # a wall, the reflex must fire.
        if avoid:
            print("[System] Avoidance Layer is SUBSUMING current state.")
            final_command = avoid
        else:
            final_command = current_intent

        return final_command

# --- Simulation ---
robot = SubsumptionRobot()
print(f"Final Motor Output: {robot.execute_cycle()}")

[System] Battery Layer is INHIBITING Navigation/Wander.
[System] Avoidance Layer is SUBSUMING current state.
Final Motor Output: ACTION: Emergency Turn (90 degrees)


#### Assignment 1: Solve the same problem with a neural net architecture (such as a Multi-Layer Perceptron)

### Exercise 2.2:  Cognitivist & Logic-Based (Symbolic / Prolog)

This architecture represents the "Thinking" paradigm. The robot maintains a Knowledge Base (KB) of facts and rules. To decide on an action, it performs a logical query: "What is the most urgent valid action given my current state?"

- Logic Rule: action(avoid) :- obstacle_detected. (If an obstacle is detected, then the action is avoid).

- The Problem: The robot must evaluate the state of the Battery, Obstacles, and Mission using logical inference.

- The Mechanism: A priority-based logical sweep.

In [7]:
class PrologCognitiveBot:
    def __init__(self):
        # Facts currently true in the World Model
        self.kb_facts = {
            "obstacle_ahead": True,
            "battery_low": False,
            "has_mission": True
        }

    def get_action(self):
        print(f"\n[Inference Engine] Checking Knowledge Base: {self.kb_facts}")

        # 1. Rule: Avoidance (Highest Logical Priority)
        if self.kb_facts["obstacle_ahead"]:
            return "AVOID (Reason: obstacle_ahead is TRUE)"

        # 2. Rule: Battery (Resource Logic)
        if self.kb_facts["battery_low"]:
            return "STOP_AND_CHARGE (Reason: battery_low is TRUE)"

        # 3. Rule: Navigation (Goal Logic)
        if self.kb_facts["has_mission"]:
            return "NAVIGATE (Reason: has_mission is TRUE)"

        # 4. Default Rule: Wander
        return "WANDER (Reason: Default behavior)"

    def update_world(self, action):
        """Simulates the robot acting and the KB updating (Ontogeny)."""
        print(f">> Resulting Action: {action}")
        if "AVOID" in action:
            self.kb_facts["obstacle_ahead"] = False
            print(">> KB Updated: Obstacle cleared.")

# --- Simulation ---
prolog_bot = PrologCognitiveBot()

# Cycle 1: Obstacle is present
action1 = prolog_bot.get_action()
prolog_bot.update_world(action1)

# Cycle 2: Obstacle cleared, mission continues
action2 = prolog_bot.get_action()
prolog_bot.update_world(action2)


[Inference Engine] Checking Knowledge Base: {'obstacle_ahead': True, 'battery_low': False, 'has_mission': True}
>> Resulting Action: AVOID (Reason: obstacle_ahead is TRUE)
>> KB Updated: Obstacle cleared.

[Inference Engine] Checking Knowledge Base: {'obstacle_ahead': False, 'battery_low': False, 'has_mission': True}
>> Resulting Action: NAVIGATE (Reason: has_mission is TRUE)


#### Assignment 2: Instead of relying on pure python-based reasoning, perform reasoning with prolog.

### Exercise 3: Hybrid Architecture (CRAM - Generalized Action Plans)

In this exercise,  a designator (a parametrized symbolic description of an entity) is implemented and resolved using reasoning and emergent perception in the context of navigation.

The Hybrid approach combines the best of both worlds:

- Symbolic Logic (High Level): Uses Prolog-like rules to decide what to do (e.g., "If battery is low, find a charger").

- Sub-symbolic Perception (Low Level): Uses Neural Networks to handle the "messy" real world (e.g., "Where exactly is the obstacle?").

In [13]:
import random

# --- SUB-SYMBOLIC LAYER (Neural Network / Sensors) ---
def neural_perception_scan():
    """Simulates a CNN detecting objects and battery."""
    return {
        "obstacle_dist": round(random.uniform(0.1, 2.0), 2),
        "battery_level": round(random.uniform(0.05, 1.0), 2)
    }

# --- SYMBOLIC LAYER (Prolog / Logic Knowledge Base) ---
def get_recovery_strategy(reason):
    strategies = {
        "LOW_BATTERY": "ACTION: Navigate to Docking Station",
        "PATH_BLOCKED": "ACTION: Compute alternative path (Left Bypass)"
    }
    return strategies.get(reason, "ACTION: Wait for assistance")

# --- HYBRID EXECUTIVE (CRAM Generalized Plan) ---
def hybrid_navigate_plan():
    print("\n[Hybrid Executive] Starting 'Navigate-to-Grasp' plan...")
    
    # 1. Perception (Sub-symbolic)
    raw_data = neural_perception_scan()
    print(f">> Perception Input: Dist={raw_data['obstacle_dist']}m, Batt={raw_data['battery_level']*100}%")

    # 2. Logic & Contextualization (The 'Architecture' at work)
    try:
        # Check Battery Logic (High-level constraint)
        if raw_data['battery_level'] < 0.1:
            strategy = get_recovery_strategy("LOW_BATTERY")
            raise Exception(strategy)

        # Check Obstacle Reflex (Subsumption-style logic within a Plan)
        if raw_data['obstacle_dist'] < 0.5:
            strategy = get_recovery_strategy("PATH_BLOCKED")
            raise Exception(strategy)

        # 3. Successful Execution
        print(">> Plan Status: Path is clear. Executing movement to target.")
        
    except Exception as e:
        # The 'Plan' catches the failure and adapts
        print(f">> Plan Adaptation: {e}")

# Run the Hybrid Cycle
for i in range(2):
    hybrid_navigate_plan()


[Hybrid Executive] Starting 'Navigate-to-Grasp' plan...
>> Perception Input: Dist=0.38m, Batt=54.0%
>> Plan Adaptation: ACTION: Compute alternative path (Left Bypass)

[Hybrid Executive] Starting 'Navigate-to-Grasp' plan...
>> Perception Input: Dist=1.17m, Batt=21.0%
>> Plan Status: Path is clear. Executing movement to target.


#### Assignment 3: Implement the perception with an actual Multi-Layer Perceptron and the reasoning with Prolog.